In [33]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from torch import optim
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import models
from torchsummary import summary
import copy
# from summary import summary

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc1 = nn.Linear(400,120)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(120,84)
        self.relu1 = nn.ReLU()
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu1(out)
        out = self.fc3(out)
        return out

model = LeNet().to(device=device)

In [35]:
# Define relevant variables for the ML task
batch_size = 100
num_classes = 10
learning_rate = 0.001

#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = '/Users/dimademler/Desktop/me-Programming/pytorch',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = '/Users/dimademler/Desktop/me-Programming/pytorch',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

def train(model,train_loader,test_loader,num_epochs,optimizer):
  total_step = len(train_loader)
  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):  
          images = images.to(device)
          labels = labels.to(device)
          
          #Forward pass
          outputs = model(images)
          loss = cost(outputs, labels)
            
          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
              
          if (i+1) % 400 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                        .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
  
def test(model, test_loader):
    # Test the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):  
            test_output = model(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))

    print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    return accuracy


In [36]:

num_epochs = 6

model = LeNet()
# baseline = LeNet(max_growth = 0)

parameters_to_prune = (
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
)



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# baseline_optim = torch.optim.Adam(baseline.parameters(), lr=learning_rate)

model_acc = []
baseline_acc = []

train(model, train_loader,test_loader,num_epochs, optimizer = optimizer)
modeloriginal= copy.deepcopy(model)


Epoch [1/6], Step [400/600], Loss: 0.0765
Epoch [2/6], Step [400/600], Loss: 0.0085
Epoch [3/6], Step [400/600], Loss: 0.0045
Epoch [4/6], Step [400/600], Loss: 0.0140
Epoch [5/6], Step [400/600], Loss: 0.0137
Epoch [6/6], Step [400/600], Loss: 0.0678


In [37]:
module = model.fc1
# print(list(module.weight))

In [38]:
print("Testing before pruning: ")

# print(LeNet)
test(model,test_loader)
vgg = models.vgg16()
# summary(vgg, (3, 224, 224))

# print(model.state_dict())




Testing before pruning: 
Test Accuracy of the model on the 10000 test images: 0.99
OrderedDict([('layer1.0.weight', tensor([[[[ 0.0780,  0.0925,  0.2535, -0.0266,  0.0510],
          [-0.1898, -0.1547,  0.0811,  0.1534,  0.1783],
          [ 0.1082, -0.1937, -0.0041, -0.0751,  0.1615],
          [ 0.0507, -0.2013, -0.1807, -0.1799, -0.0739],
          [ 0.1296, -0.0833, -0.0534,  0.0724, -0.2185]]],


        [[[-0.0978, -0.1907, -0.0294, -0.2541, -0.2660],
          [ 0.0935, -0.0230, -0.1997, -0.1020,  0.0864],
          [-0.0774, -0.0897,  0.0165,  0.2444,  0.2287],
          [-0.1112,  0.2113,  0.1765, -0.0452, -0.1106],
          [ 0.0113, -0.0447,  0.0612,  0.0520,  0.1021]]],


        [[[ 0.2650, -0.0685,  0.0216, -0.0378,  0.0835],
          [ 0.1460,  0.1539, -0.0546,  0.0831, -0.0410],
          [ 0.0427, -0.0256,  0.1388, -0.1371, -0.0353],
          [-0.0097, -0.0101, -0.1266, -0.0485, -0.1070],
          [ 0.1089, -0.1521, -0.1577, -0.1783, -0.1391]]],


        [[[-0.014

In [39]:
modelcopy2= copy.deepcopy(model)

parameters_to_prune2 = (
    (modelcopy2.fc1, 'weight'),
    (modelcopy2.fc2, 'weight'),
)
prune.global_unstructured(
    parameters_to_prune2,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)


num_timespruned=30


# run pruning once

for module,name in parameters_to_prune2:
  # torch.nn.utils.prune.random_unstructured(module, name, num_prunedfloat)
  torch.nn.utils.prune.global_unstructured(module, pruning_method, importance_scores=None)




for i in range(num_timespruned):
  state = modelcopy2.state_dict()
  torch.save(state, "modelcopy2_state.pth")

  modelretrain=  LeNet().to(device=device)
  modelretrain.load_state_dict(torch.load("modelcopy2_state.pth"))


# fc1mask=modelcopy2.fc1
# fc2mask=modelcopy2.fc2





NameError: name 'pruning_method' is not defined

In [ ]:
modelcopy= copy.deepcopy(model)

# max_growth = 30
num_prunedfloat=0.2


parameters_to_prune = (
    (modelcopy.fc1, 'weight'),
    (modelcopy.fc2, 'weight'),
)

for module,name in parameters_to_prune:
  torch.nn.utils.prune.random_unstructured(module, name, num_prunedfloat)


print("Testing after pruning: ")
test(modelcopy,test_loader)

module2 = modelcopy.fc2
# print(list(module2.named_buffers()))
# print(list(module2.weight))
# vgg = models.vgg16()
# summary(vgg, (3, 224, 224))




Testing after pruning: 
Test Accuracy of the model on the 10000 test images: 1.00
[('weight_mask', tensor([[1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 0., 1.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 0., 1.],
        [0., 1., 1.,  ..., 1., 1., 0.]]))]


In [8]:
max_growth = 20
num_epochs = 10
gamma = 10 #num new nodes on first growth

model = LeNet(max_growth = max_growth)
baseline = LeNet(max_growth = 0)

parameters_to_prune = (
#    (model.conv1, 'weight'),
#    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
#    (model.fc3, 'weight'),
)
for module,name in parameters_to_prune:
  torch.nn.utils.prune.random_structured(module, name, max_growth,  0)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
baseline_optim = torch.optim.Adam(baseline.parameters(), lr=learning_rate)

model_acc = []
baseline_acc = []
for i in range(5):
  #grow 1/i*growth_factor params total:
  
  #for module,name in parameters_to_prune:
    #grow
  
    

  


  train(model, train_loader,test_loader,num_epochs = 5, optimizer = optimizer)
  # train(baseline, train_loader,test_loader,num_epochs = 5, optimizer = baseline_optim)
  # model_acc.append(test(model, test_loader))
  baseline_acc.append(test(baseline, test_loader))

  plt.plot(np.arange(len(model_acc)), model_acc, label = 'Adaptive Model')
  plt.plot(np.arange(len(baseline_acc)), baseline_acc, label = 'Baseline')
  plt.show()

  prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.10+(-1*i*.01),
  )

Epoch [1/5], Step [400/600], Loss: 0.0537
Epoch [2/5], Step [400/600], Loss: 0.0773
Epoch [3/5], Step [400/600], Loss: 0.0305
Epoch [4/5], Step [400/600], Loss: 0.0182


KeyboardInterrupt: 

In [ ]:
# parameters_to_prune2 = (
#     (model.fc1, 'weight'),
#     (model.fc2, 'weight'),
# )
# prune.global_unstructured(
#     parameters_to_prune2,
#     pruning_method=prune.L1Unstructured,
#     amount=0.5
# )
# # run pruning once
# # prune.global_unstructured.pruning_method
#   # torch.nn.utils.prune.random_unstructured(module, name, num_prunedfloat)
# prune.global_unstructured(parameters_to_prune2,prune.L1Unstructured, importance_scores=None, amount=0.5)